In [ ]:
import cv2
import numpy as np
import os

# Load HAAR face classifier
face_classifier = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

def face_extractor(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    if len(faces) == 0:
        return None

    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]

    return cropped_face

def collect_face_samples(name, data_path='D://Projects//ML Code//Computer Vision//ML-workshop-ec2-CV-security-master//samyak'):
    cap = cv2.VideoCapture(0)
    count = 0

    while True:
        ret, frame = cap.read()
        detected_face = face_extractor(frame)

        if detected_face is not None:
            count += 1
            face = cv2.resize(detected_face, (200, 200))
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            file_name_path = os.path.join(data_path, f'{name}_image{count}.jpg')
            cv2.imwrite(file_name_path, face)

            cv2.putText(face, str(count), (50, 50), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
            cv2.imshow('Face Cropper', face)
        else:
            print("Face not found")
            pass

        if cv2.waitKey(1) == 13 or count == 100:
            break

    cap.release()
    cv2.destroyAllWindows()
    print("Collecting Samples Complete")

def train_face_recognition(data_path='D://Projects//ML Code//Computer Vision//ML-workshop-ec2-CV-security-master//samyak'):
    onlyfiles = [f for f in os.listdir(data_path) if os.path.isfile(os.path.join(data_path, f))]

    Training_Data, Labels = [], []

    for i, file in enumerate(onlyfiles):
        image_path = os.path.join(data_path, file)
        print("Loading image:", image_path)
        images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
        
        if images is not None:
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(i)
            print("Image loaded successfully.")
        else:
            print("Error loading image:", image_path)

    print("Number of training images loaded:", len(Training_Data))

    Labels = np.asarray(Labels, dtype=np.int32)
    model = cv2.face.LBPHFaceRecognizer_create() 

    model.train(np.asarray(Training_Data), np.asarray(Labels))
    print("Model trained successfully")

    return model

def recognize_face(model, name):
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        image, face = face_detector(frame)

        try:
            face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)

            results = model.predict(face)
            print(results)
            if results[1] < 500:
                confidence = int(100 * (1 - (results[1])/400))
                display_string = f'{confidence}% Confident it is {name}'

            cv2.putText(image, display_string, (100, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 120, 150), 2)

            if confidence > 80:
                cv2.putText(image, f'Hey {name}', (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 255, 0), 2)
                cv2.imshow('Face Recognition', image )
            else:
                cv2.putText(image, "I don't know", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
                cv2.imshow('Face Recognition', image )

        except Exception as e:
            print(f"Error Recognising Face: {e}")
            cv2.putText(image, "Error Recognising Face", (130, 120), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.putText(image, "Locked", (250, 450), cv2.FONT_HERSHEY_COMPLEX, 1, (0, 0, 255), 2)
            cv2.imshow('Face Recognition', image )
            pass

        if cv2.waitKey(1) == 13:
            break

    cap.release()
    cv2.destroyAllWindows()

def face_detector(img, size=0.5):
    # Convert image to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)
    
    if len(faces) == 0:
        return img, []

    for (x, y, w, h) in faces:
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 255), 2)
        roi = img[y:y+h, x:x+w]
        roi = cv2.resize(roi, (200, 200))
        
    return img, roi

if __name__ == "__main__":
    # Get user's name
    Name = input("Enter Your name here:")

    # Step 1: Collect face samples
    collect_face_samples(Name)

    # Step 2: Train face recognition model
    model = train_face_recognition()

    # Step 3: Recognize face using the trained model
    recognize_face(model, Name)
